In [1]:
%pylab inline
import os
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


[!img](http://4.bp.blogspot.com/-Ocn91gy9E5M/UBc1emNFh9I/AAAAAAAAAVY/E6vmshsdeZ4/s1600/Project+log+(BU_HD)+(1).png)

# Mouse

In [2]:
sheet_name = 'mm10'
excel_df = pd.read_excel('../data/datasets_info.xlsx', sheet_name=sheet_name)
excel_df

,GEO,SRA,RIBO sample,RNA sample,read lengths,offsets,RIBO adapter,RNA adapter,RIBO strandedness,RNA strandedness
0,GSE22001,SRP003554,SRX026871,SRX026872,"28,29,30","12,12,12",TCGTATG,TCGTATG,forward,forward
1,GSE72064,SRP062407,SRX1149649,SRX1149668,"28,29,30,31","12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
2,GSE84112,SRP078005,SRX1900396,SRX1900402,"26,27,28,29,30","12,12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
3,GSE89011,SRP091889,SRX2255510,SRX2255511,"26,27,28,29,30","12,12,12,12,12",AGATCGGAAGAGCACACGTCT,AGATCGGAAGAGCACACGTCT,forward,forward
4,GSE102890,SRP115915,SRX3110803,SRX3110807,"29,30,31,32,33,34","12,12,12,13,13,13",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
5,GSE99787,SRP108787,SRX2893684,SRX2893692,"25,26,27,28","12,10,11,12",None,None,forward,reverse


In [3]:
config_out_dir = '/auto/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/spectre/configs'
input_dir_root = '/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/'
intron_bed = '/home/cmb-panasas2/skchoudh/genomes/mm10/annotation/gencode.vM11.gffutils.intron.bed'
chrom_sizes = '/home/cmb-panasas2/skchoudh/genomes/mm10/fasta/mm10.chrom.sizes'
genome_fasta = '/home/cmb-panasas2/skchoudh/genomes/mm10/fasta/mm10.fa'
gtf = '/home/cmb-panasas2/skchoudh/genomes/mm10/annotation/gencode.vM11.annotation.gtf'
star_index = '/home/cmb-panasas2/skchoudh/genomes/mm10/star_annotated/'

In [4]:
def convert_strandedness(strandedness):
    strandedness_map = {'forward': 'fr-secondstrand',
                         'reverse': 'fr-firststrand',
                         'no': 'fr-unstranded'}
    return strandedness_map[strandedness]

In [5]:
offset_file_map = {}
for index, row in excel_df.iterrows():
    read_lengths = list(row['read lengths'].strip().split(','))
    offsets = row['offsets'].strip().split(',')
    
    zipped = zip(read_lengths, offsets)
    targets = (',').join([(':').join(x) for x in zipped])
    srp = row['SRA']
    print(srp)
    ribo_sample = row['RIBO sample']
    rna_sample = row['RIBO sample']
    input_dir = os.path.join(input_dir_root, srp+'_mouse')
    rna_strand = convert_strandedness(row['RNA strandedness'])
    ribo_strand = convert_strandedness(row['RIBO strandedness'])
    config = dedent('''
    INPUT_DIR = '{}'
    GTF = '{}'
    RIBO_STRAND_CUFFLINKS = '{}'
    RNA_STRAND_CUFFLINKS = '{}'
    OFFSETS = '"{}"'
    ''').format(input_dir, gtf, ribo_strand, rna_strand, targets)
    with open(os.path.join(config_out_dir, '{}_mouse.py'.format(srp)), 'w') as fh:
        fh.write(config)
    

SRP003554
SRP062407
SRP078005
SRP091889
SRP115915
SRP108787


# Human

In [6]:
assembly_common_name_map = {'mm10': 'mouse', 'hg38': 'human'}

sheet_name = 'hg38'
excel_df = pd.read_excel('../data/datasets_info.xlsx', sheet_name=sheet_name)
excel_df

,GEO,SRA,RIBO sample,RNA sample,read lengths,offsets,RIBO adapter,RNA adapter,RIBO strandedness,RNA strandedness,AUC(RiboCop/RiboCode/RiboTaper/ORFscore),metagene_coherence_ribocop
0,GSE35469,SRP010679,SRX118286,SRX118285,"28,29,30","12,13,13",CTGTAGGCAC,CTGTAGGCAC,forward,forward,NaN,NaN
1,NaN,SRP029589,SRX345309,SRX345311,"29,30,32","12,12,13",CTGTAGGCACCATCAAT,CTGTAGGCACCATCAAT,forward,forward,NaN,NaN
2,GSE73136,SRP063852,SRX1254413,"SRX426378 (SRP028887, GSE49831)","28,29,30","12,12,12",None,None,forward,forward,NaN,NaN
3,GSE94454,SRP098789,SRX2536421,SRX2536426,"28,30","12,13",CTGTAGGCACCATCAAT,CTGTAGGCACCATCAAT,forward,forward,NaN,NaN
4,GSE96716,SRP102021,SRX2647167,SRX2647164,"28,29,30,31","12,12,12,12",TCGTATGCCGTCTTCTGCTTG,None,forward,forward,NaN,NaN
5,GSE87328,SRP090415,SRX2189157,SRX2189154,"26,27,28,29,30","12,12,12,12,12",CTGTAGGCACCATCAAT,AGATCGGAAGAGCACAC,forward,reverse,NaN,NaN
6,GSE89183,SRP092068,SRX2268404,SRX2268396,"26,27,28,29,30","12,12,12,12,12",AGATCGGAAGAGCACACGTCT,None,forward,reverse,NaN,NaN
7,GSE96714,SRP102020,SRX2647162,SRX2647160,"29,30","12,12",TCGTATGCCGTCTTCTGCTTG,None,forward,no,NaN,NaN


In [7]:
config_out_dir = '/auto/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/spectre/configs'
input_dir_root = '/home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/'
gtf = '/home/cmb-panasas2/skchoudh/genomes/hg38/annotation/gencode.v25.annotation.gtf'


In [8]:
offset_file_map = {}
for index, row in excel_df.iterrows():
    read_lengths = list(row['read lengths'].strip().split(','))
    offsets = row['offsets'].strip().split(',')
    
    zipped = zip(read_lengths, offsets)
    targets = (',').join([(':').join(x) for x in zipped])
    srp = row['SRA']
    print(srp)
    ribo_sample = row['RIBO sample']
    rna_sample = row['RIBO sample']
    input_dir = os.path.join(input_dir_root, srp+'_human')
    rna_strand = convert_strandedness(row['RNA strandedness'])
    ribo_strand = convert_strandedness(row['RIBO strandedness'])
    config = dedent('''
    INPUT_DIR = '{}'
    GTF = '{}'
    RIBO_STRAND_CUFFLINKS = '{}'
    RNA_STRAND_CUFFLINKS = '{}'
    OFFSETS = '"{}"'
    ''').format(input_dir, gtf, ribo_strand, rna_strand, targets)
    with open(os.path.join(config_out_dir, '{}_human.py'.format(srp)), 'w') as fh:
        fh.write(config)
    

SRP010679
SRP029589
SRP063852
SRP098789
SRP102021
SRP090415
SRP092068
SRP102020
